# Writing your own ROS2 Code

The goal here is to write your first ROS2 code from scratch, subscribing to the battery (using what already exists in ROS2Lib) and then writing - from scratch - a code that will subscribe to the proximity sensor.  Finally, you will combine that with the twist code in ROS2Lib to make a car that keeps a safe distance from anything in front of it.

## Subscribing to the battery

First set up your domain ID and then see if you can echo it

In [ ]:
import os
os.environ['ROS_DOMAIN_ID'] = "0"

In [ ]:
!ros2 topic echo /rogers/battery_state

Now to write that in python.  First, we need to find the syntax - what type of message is getting sent?  You can use *ros2 topic info* to figure that one out (or go [here](https://iroboteducation.github.io/create3_docs/api/ros2/)).

In [ ]:
!ros2 topic info /rogers/battery_state

So we know there is one publisher (the Create) and some subscribers (one of them is us).  We therefore know that we will need
``` python
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import BatteryState
from rclpy.qos import qos_profile_sensor_data
```
with the last line (*qos*) defines if you just want to get the latest or not.  So now we need to create a battery class, using that message type, initiate a subscriber with that message type (*BatteryState*) and a callback for when the Create sends out the next battery level message, and update a variable in the class (*percentage*) when that happens.

In [ ]:
import rclpy
from rclpy.node import Node
import time, os

from rclpy.qos import qos_profile_sensor_data
from sensor_msgs.msg import BatteryState

class Battery(Node):
    '''
    Set up a node that lets you subscribe to the battery level
    '''
    def __init__(self, namespace = '/rogers'):   
        '''
        define the node and set up the subscriber
        '''
        super().__init__('battery_subscription') #name it anything you want - this is what will appear in the ros topic list
        
        self.subscription = self.create_subscription(BatteryState, namespace + '/battery_state', self.callback, qos_profile_sensor_data)
        self.done = False
        
    def callback(self, msg: BatteryState):
        self.charge = 100*msg.percentage
        self.done = True   # tell the parent program you are done - you have data

os.environ['ROS_DOMAIN_ID'] = "0"
create = '/rogers'

rclpy.init()
battery_subscriber = Battery(create)
rclpy.spin_once(battery_subscriber)  # start it up once

while not battery_subscriber.done:  # keep running until you get a result
    print('...', end = '')
    rclpy.spin_once(battery_subscriber)

print('%s battery at %0.1f ' % (create, battery_subscriber.charge))

rclpy.shutdown()   # shut everything down

## Subscribing to the infrared proximity sensor

Now we have to do the same thing for the ir proximity sensor.  You can do a ros2 topic list and take a guess at which topic will give you the information.  Then get the message type and build a subscriber by copying the one above but replacing everything for the ir instead of battery.  To parse the incoming message, look at how I did the battery one above and compare it to what you read with *ros2 topic echo /rogers/battery_state* and see if you can guess it.  To get the labels of all the 5 ir sensors, I used this in the callback (note this builds an array for every separate reading in the message).
```python
self.names = [h.header.frame_id for h in msg.readings]
```

## Publishing the desired twist

If we look back at the code you used for the Airtable or for the keyboard control, you know that we can set the velocity of the robot with the twist call.  Grab that class from the ROS2Lib file (in Subs) and make sure that works on its own as well (try telling it to move forward for a second and then stop). If you are uncertian how to publish, look at CreateLib and see how it is done there.

## Going backwards

You probably want to reset the safety parameter to let the Create back up.  Here is a code that does that using a service - try integrating that into your code.

In [ ]:
# see https://iroboteducation.github.io/create3_docs/api/safety/

import rclpy
from rclpy.node import Node

from rcl_interfaces.srv import SetParameters
from rcl_interfaces.msg import ParameterType, ParameterValue, Parameter

import os

class Safety(Node):
    def __init__(self, namespace = '/Picard'):
    
        super().__init__('linefollow')
        self.params_client = self.create_client(SetParameters, namespace + '/motion_control/set_parameters')
    
    def Set(self, task):
        '''
        define the parameter that we wish to append. In this case we want the safety override parameter to be 'full'
        options are done, backup_only, full'''
        
        options = ['full','backup_only','none']
        if not task in options:
            return None
        request = SetParameters.Request()
        param = Parameter() 
        param.name = "safety_override"
        param.value.type = ParameterType.PARAMETER_STRING
        param.value.string_value = task
        request.parameters.append(param)
        
        # wait for a service client to be available then send that appended parameter
        print('setting safety to ' + task,end='')
        self.params_client.wait_for_service()
        print(' ... ',end='')
        self.future = self.params_client.call_async(request)
        print('done')

    def AllOn(self):
        self.Set('none')
        
    def Backup(self):
        self.Set('backup_only')
        
    def AllOff(self):
        self.Set('full')

os.environ['ROS_DOMAIN_ID'] = "7"
create = '/Buzz'

rclpy.init()
safety = Safety(create)
safety.Backup()
rclpy.shutdown()

## Bringing it all together

Now the deal is have the callback change the motor speed - so that it is proportional to the IR reading (maybe average the front left and front right or just use one of them).  That is, when you get a callback, call twist.  To succeed, that means you will have to initialize a twist_publisher in the init of the ir class, and call it in the ir callback.  The real magic is figuring out the equation for calculating motor speed as a function of ir reading.